In [1]:
%jsroot on

In [2]:
#include "/data/d1/zhuhy/lanzhou2022/getdatata/include/headerInclude.hh"

In [3]:
TCanvas *c1=new TCanvas("c1","c1");

In [4]:
#define _N 3

  string ss;
  Double_t aa, bbb, e, dedx;
  i=0;
  Double_t E0=0,Etem=0,Esmall=0,E2=0,E=0,ddE=0,x=0,dx=0.1;
//E0为能量总值，E为能量实时记录值，E1为穿透第一块硅时的能量，E2为穿透第二块时的能量
  Double_t se4He[16][_N][3];

In [5]:
Double_t xxs,ad3_thick[16];

char str_thick[100]="/home/zhuhy/JNBook/Experiment2022/taf/csicali/txt/taf_ad0_thickreal.txt";
ifstream in_thick(str_thick);
  if(in_thick.is_open()){
//    getline(in,ss);
    Int_t i=0; 
    while(i<16)
    {
      in_thick>>xxs>>ad3_thick[i];
      i++;
    }
  }
  in_thick.close();

In [6]:
   Double_t mean0[17];
   Double_t mean1[17];
   Double_t mean2[17];
   Double_t sigma0[17];
   Double_t sigma1[17];
   Double_t sigma2[17];

Double_t dE0[17];
Double_t dE1[17];
Double_t dE2[17];

Double_t k[17];
Double_t b[17];
Double_t r[17];

ifstream read1;
read1.open("/home/zhuhy/JNBook/Experiment2022/taf/cali/alphacali/calitxt/taf_ad5.txt", ios::in);


        if(!read1.is_open()){
                cout<<" Cannot find normParas" <<endl;
                return;
        }
        Int_t line = 0;
        Double_t tmp;
        while(!read1.eof()&&line<17){
                read1>>mean0[line]>>sigma0[line]>>mean1[line]>>sigma2[line]>>mean2[line]>>sigma2[line];
                line++;
        }
        read1.close();

In [7]:
Double_t E1real[16],E2real[16],E3real[16];

In [8]:
TString  str_4He[2];
str_4He[0]=TString::Format("/home/zhuhy/JNBook/15C_Etheta/TAT0_datacheck/PID/txt/4He-Si.txt");
str_4He[1]=TString::Format("/home/zhuhy/JNBook/15C_Etheta/TAT0_datacheck/PID/txt/4He-Al.txt");

TGraph *g_4He[2];
  ifstream in_4He(str_4He[0].Data());

  g_4He[0]=new TGraph;//定义新的二维散点图(TGraph) g1
  if(in_4He.is_open()){//检查文件是否存在
//    getline(in,ss);
    i=0; 
    while(!in_4He.eof())
    {
      in_4He>>aa>>bbb>>e>>dedx>>aa>>bbb>>aa>>bbb>>aa>>bbb>>aa>>bbb;
      // 对照数据文件进行理解。一列一列地读入，提取第3、4列数据。
      g_4He[0]->SetPoint(i++, e*4,dedx);//添加数据点到图中，能量单位为MeV/u，对C需乘以12.    
    }
  }
  in_4He.close();
  ifstream in1_4He(str_4He[1].Data());
  g_4He[1]=new TGraph;
  if(in1_4He.is_open()){
//    getline(in1,ss);
    i=0;
    while(!in1_4He.eof())
    {
      in1_4He>>aa>>bbb>>e>>dedx>>aa>>bbb>>aa>>bbb>>aa>>bbb>>aa>>bbb;
      g_4He[1]->SetPoint(i++, e*4,dedx);
    }
  }
  in1_4He.close();





 Double_t d[_N-1];
/*
 //d[0]=ad3_thick[stripnum];
 //d[1]=1;
 //d[2]=ad3_thick[stripnum];
 Int_t k; //kind  0:Si 1:Al 2:Mylar
 Int_t num_ad3=0;
Int_t ui;
*/

for(Int_t num_ad3=0;num_ad3<16;num_ad3++){
//cout<<ad3_thick[num_ad3]<<" : "<<ad4_thick[num_ad4]<<'\n';
     
 d[0]=0.5*ad3_thick[num_ad3]/150.;
 d[1]=ad3_thick[num_ad3];
 for(i=1;i<(_N-1);i++)d[i]=d[i]+d[i-1];
 for(i=-1;i<3;x=0)   //开始3次模拟
  {      
  i++;
  if(i>=3)break;
  x=0;
  if(i==0)E0=E=5.15659;
  if(i==1)E0=E=5.48556;
  if(i==2)E0=E=5.80477;

  for(Int_t j=0;j<_N;j++)se4He[num_ad3][j][i]=0;
  se4He[num_ad3][_N-1][i]=E0;
  Etem=E0;
Int_t   kk=1;
//cout<<" Y1"<<'\n';
     x=0.;
  while(1)
    {
      
 //     if(ui%500==0)cout<<x<<"  ";
     if((fabs(x-d[0])<=(dx/100.)||(x>=0&&x<d[0]))&&E<=0){se4He[num_ad3][0][i]=Etem;Etem=0;break;}
     else if(fabs(x-d[0])<=(dx/100.)&&E>0){if(kk!=0){se4He[num_ad3][0][i]=E0-E;Etem=E;}kk=0;}

     else if(((fabs(x-d[1])>(dx/100.)&&x>d[1])&&E<=0)){se4He[num_ad3][1][i]=Etem;Etem=0;break;}
     else if(((fabs(x-d[1])>(dx/100.)&&x>d[1])&&E>0)){se4He[num_ad3][1][i]=Etem-E;Etem=0;break;}
     else if(fabs(x-d[1])<=(dx/100.)&&E>0){if(kk!=1){se4He[num_ad3][1][i]=Etem-E;Etem=E;}kk=1;}
     
     ddE=g_4He[kk]->Eval(E);
     if(kk==0){
     E=E-ddE*dx;
     x=x+dx;
     }
     else {
     E=E-ddE*dx/50.;
     x=x+dx/50.;
     }
    }
//     cout<<"Num="<<i<<'\t'<<"E="<<E<<'\t'<<"dE_Si="<<sep[num_ad3][1][i]<<'\t'<<"dE_CsI="<<sep[num_ad3][3][i]<<'\n';
     
//          cout<<"peaknum="<<i<<" strip="<<num_ad3<<'\t'<<"Ereal="<<se4He[num_ad3][1][i]<<'\n';  
  } 
}

//cout<<"*************************************************************************************************************"<<'\n';



  for(Int_t i0=0;i0<3;i0++){
      for(Int_t j0=0;j0<16;j0++){
        if(i0==0)E1real[j0]=se4He[j0][1][i0];
        if(i0==1)E2real[j0]=se4He[j0][1][i0];
        if(i0==2)E3real[j0]=se4He[j0][1][i0];   
      }

  }


In [9]:
TGraph *gfit[17];
TGraph *dE[3];
TGraph *kk=new TGraph();
TGraph *bb=new TGraph();
for(Int_t ii=0;ii<17;ii++)gfit[ii]= new TGraph();
for(Int_t ii=0;ii<3;ii++)dE[ii]= new TGraph();

for(Int_t ii=0;ii<16;ii++){
    gfit[ii]->SetPoint(0,mean0[ii],E1real[ii]);
    gfit[ii]->SetPoint(1,mean1[ii],E2real[ii]);
    gfit[ii]->SetPoint(2,mean2[ii],E3real[ii]);
    TFitResultPtr r1 =gfit[ii]->Fit("pol1","QS");
    k[ii]=r1->Value(1);
    b[ii]=r1->Value(0);
    kk->SetPoint(ii,ii,k[ii]);
    bb->SetPoint(ii,ii,b[ii]);
    //r[ii]=r1->Chisquare();
    dE0[ii]=E1real[ii]-(mean0[ii]*k[ii]+b[ii]);
    dE1[ii]=E2real[ii]-(mean1[ii]*k[ii]+b[ii]);
    dE2[ii]=E3real[ii]-(mean2[ii]*k[ii]+b[ii]);
    dE[0]->SetPoint(ii,ii,dE0[ii]);
    dE[1]->SetPoint(ii,ii,dE1[ii]);
    dE[2]->SetPoint(ii,ii,dE2[ii]);
}

Error in <TDecompChol::Decompose()>: matrix not positive definite
Error in <TDecompChol::Solve()>: Decomposition failed
Error in <TLinearFitter::Eval>: Matrix inversion failed
Warning in <Minimize>: TLinearFitter failed in finding the solution
Error in <TDecompChol::Decompose()>: matrix not positive definite
Error in <TDecompChol::Solve()>: Decomposition failed
Error in <TLinearFitter::Eval>: Matrix inversion failed
Warning in <Minimize>: TLinearFitter failed in finding the solution
Error in <TDecompChol::Decompose()>: matrix not positive definite
Error in <TDecompChol::Solve()>: Decomposition failed
Error in <TLinearFitter::Eval>: Matrix inversion failed
Warning in <Minimize>: TLinearFitter failed in finding the solution
Error in <TDecompChol::Decompose()>: matrix not positive definite
Error in <TDecompChol::Solve()>: Decomposition failed
Error in <TLinearFitter::Eval>: Matrix inversion failed
Warning in <Minimize>: TLinearFitter failed in finding the solution
Error in <TDecompChol::D

In [10]:
c1->Clear();
gfit[0]->SetMarkerStyle(3);
gfit[0]->Draw("ap");
c1->Draw();

In [11]:
c1->Clear();
gfit[15]->SetMarkerStyle(3);
gfit[15]->Draw("ap");
c1->Draw();

In [12]:
c1->Clear();
kk->SetMarkerStyle(3);
kk->Draw("ap");
c1->Draw();

In [13]:
c1->Clear();
bb->SetMarkerStyle(3);
bb->Draw("ap");
c1->Draw();

In [14]:
c1->Clear();
dE[0]->SetMarkerStyle(3);
dE[0]->Draw("ap");
c1->Draw();

In [15]:
c1->Clear();
dE[1]->SetMarkerStyle(3);
dE[1]->Draw("ap");
c1->Draw();

In [16]:
c1->Clear();
dE[2]->SetMarkerStyle(3);
dE[2]->Draw("ap");
c1->Draw();

In [17]:
        ofstream write1;
        TString fileName = TString::Format("/home/zhuhy/JNBook/Experiment2022/taf/cali/alphacali/calitxt/taf_ad5_kb.txt");
        write1.open(fileName.Data(), ios::out);
        for(Int_t i=0; i<4; i++)write1<< 0<< "\t" <<0<<"\t" <<endl;
        for(Int_t i=4; i<15; i++)write1<< k[i]<< "\t" <<b[i]<<"\t" <<endl;
        write1<< 0<< "\t" <<0<<"\t" <<endl;
        write1.close();